# Spelling Corrector

**Problem Statement:** 

The goal of the problem is to build a model which can check the spelling of a word and if the spelling is incorrect it can suggest the correct spelling.

**Solution:**

We will implement `Norvig’s Spell Checker` algorithm to solve this. The solution has following steps:

- Load the correct text corpus from any data source
- Define below utilities:
    - Generate all the combination of words which are 1 edit distance away from given word
    - Generate all the combination of words which are 2 edit distance away from given word
    - Generate all known words of a given word, known words means the word which is present in correct text corpus
- Define all possible correction of a word as `known_words([word]) or known_words(edit_one(word)) or known_words(edit_two(word)) or [word]`
- For all possible correction return the word which has highest probability of occurrence.

In [14]:
# import libraries
import re
from collections import Counter

In [18]:
# load the correct text corpus
correct_text = open('big.txt').read()

# tokenize the correct text corpus 
tokens = re.findall(r'\w+', correct_text)

# create global dictionary of all correct words
all_words = Counter(tokens)

In [35]:
# generate all possible combination of words which are 1 edit distance away from given word

def edit_one(word):
    alphabets = 'abcdefghijklmnopqrstuvwxyz'
    
    splits = []
    for i in range(0, len(word)):
        left = word[:i]
        right = word[i:]
        splits.append([left, right])

    deletes = []
    for left, right in splits:
        if len(right) > 0:
            deletes.append(left + right[1:])

    inserts = []
    for left, right in splits:
        for c in alphabets:
            inserts.append(left + c + right)

    replaces = []
    for left, right in splits:
        if len(right) > 0:
            for c in alphabets:
                replaces.append(left + c + right[1:])

    transposes = []
    for left, right in splits:
        if len(right) > 1:
            transposes.append(left + right[1] + right[0] + right[2:])

    return set(deletes + inserts + replaces + transposes)

In [20]:
# generate all possible combination of words which are 2 edit distance away from given word

def edit_two(word):
    combinations = []

    for e1 in edit_one(word):
        for e2 in edit_one(e1):
            combinations.append(e2)

    return set(combinations)

In [21]:
# generate all known words 

def known_words(words):
    known = []

    for word in words:
        if word in all_words:
            known.append(word)

    return set(known)

In [29]:
# generate all possible combinations of given word

def possible_corrections(word):
    return (known_words([word]) or known_words(edit_one(word)) or known_words(edit_two(word)) or [word])

In [23]:
# define probability of correct word

def prob(word, N=sum(all_words.values())):
    return all_words[word] / N

In [58]:
def spell_corrector(word):
    correct_word = max(possible_corrections(word), key=prob)
    
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [59]:
spell_corrector('monney')

'Did you mean money?'

In [38]:
spell_corrector('money')

'Correct spelling.'

In [53]:
spell_corrector('flower')

'Correct spelling.'